## -----------Start M2-Week 9 Assignment/31st Oct 2016------------------------------
<li><strong>Title:</strong> Module 2 / Week 9: Other Classifiers(Naive Bayes)   
<li><strong>Purpose:</strong>Naive Bayes with Text Extraction 
<li><strong>Name:</strong>RAJESH ASHOK JAISWAL UIN:669012329
<li><strong>DATED:</strong>10/31/2016

In [1]:
#Here I am going to import all required parameters and classifier
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

#Here I am going to store text data from "SMSSpamCollection" file into Local dataframe'Tdata'

In [2]:
#Will Separate  data into two columns 'Spam' and 'Txt'
Tdata= pd.read_csv("SMSSpamCollection",sep='\t', names=['Spam', 'Txt'])

In [3]:
#Will Display Data
Tdata.head()

,Spam,Txt
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
Tdata.Spam.value_counts()

ham     4825
spam     747
Name: Spam, dtype: int64

In [5]:
Tdata.Spam.value_counts().plot(kind='barh')

In [6]:
#will check is there any missing values
[col for col in Tdata.columns 
     if Tdata[col].isnull().any()]

[]

#### As per the out[8] result above data not have any missing values.

In [7]:
# Now I am going to run stopword and TFIDF Vectorizer to remove stop word if there any
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

#### In this section I am going to create dummies for column Spam for proper result calculation.
#### I am going to create Spam column with entries with '0,1' numeric values.
#### 0= will denote sms is 'Ham' and 1=will denote sms is 'Spam'

In [8]:
# here (0,1)==(ham,Spam)
Tdata['Spam'] = pd.get_dummies(Tdata.Spam)['spam']

In [9]:
#let see values of Spam Column after dummy creation
#in this case our dependent variable will be as 0 (sms 'ham') or 1 (sms is 'spam')
Tdata.Spam

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
5       1.0
6       0.0
7       0.0
8       1.0
9       1.0
10      0.0
11      1.0
12      1.0
13      0.0
14      0.0
15      1.0
16      0.0
17      0.0
18      0.0
19      1.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
5542    0.0
5543    0.0
5544    0.0
5545    0.0
5546    0.0
5547    1.0
5548    0.0
5549    0.0
5550    0.0
5551    0.0
5552    0.0
5553    0.0
5554    0.0
5555    0.0
5556    0.0
5557    0.0
5558    0.0
5559    0.0
5560    0.0
5561    0.0
5562    0.0
5563    0.0
5564    0.0
5565    0.0
5566    1.0
5567    1.0
5568    0.0
5569    0.0
5570    0.0
5571    0.0
Name: Spam, dtype: float64

In [10]:
# As we are predicting value of depenedent 'Spam' column will store it in Ty
Ty = Tdata.Spam

In [11]:
#will se value stroed in Ty
Ty.head()

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: Spam, dtype: float64

In [12]:
#Now will store independent variable in Tx and convert Tdata.Txt to features
Tx= vectorizer.fit_transform(Tdata.Txt)

In [13]:
#let's see values in Tx feature
Tx

<5572x8587 sparse matrix of type '<type 'numpy.float64'>'
	with 47427 stored elements in Compressed Sparse Row format>

In [14]:
#Dispalay shape of Ty and Tx

print Ty.shape
print Tx.shape

(5572,)
(5572, 8587)


In [15]:
#Test Train Split as usual using 0.2 test split
X_train, X_test,y_train, y_test = train_test_split(Tx, Ty, test_size=0.2,random_state=42)

In [16]:
X_train.shape

(4457, 8587)

In [17]:
X_test.shape

(1115, 8587)

### ------------------------------Naive Bayes Classifier Application-----------------------------

In machine learning, naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features.

In [18]:
#we will train a naive_bayes classifier
Tclf = naive_bayes.MultinomialNB()
Tclf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Accuracy Score

In [19]:
#We can test our model's accuracy like this:
roc_auc_score(y_test, Tclf.predict_proba(X_test)[:,1])

0.98471521669654161

## ------------------------------------------------------------------------------

## Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
model = RandomForestRegressor(100, oob_score=True, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
print "C-stat: ", roc_auc_score(y_train, model.oob_prediction_)

C-stat:  0.980200261561


## ------------------------------------------------------------------------------

## Linear Regression

In [22]:
np.random.seed(100)
from sklearn.linear_model import LinearRegression
modell = LinearRegression()
modell.fit(X_train, y_train)
print "R^2:", modell.score(X_test, y_test).round(2)

R^2: 0.76


## ------------------------------------------------------------------------------

## Decision Tree

In [23]:
from sklearn.tree import DecisionTreeRegressor
modelt = DecisionTreeRegressor()
modelt.fit(X_train, y_train)
print "R^2:", modelt.score(X_test, y_test).round(2)

R^2: 0.78


## ---------------------New entry testing-------------------------------------

## Naive bayes

In [24]:
sms_spam_review=np.array(["50 free entries for Qatar 2022 football world cup in 2 dollar"])

sms_spam_review_vector = vectorizer.transform(sms_spam_review)

print Tclf.predict(sms_spam_review_vector)

[ 0.]


## Random Forest

In [25]:
print model.predict(sms_spam_review_vector)

[ 0.28]


## Linear Regression

In [26]:
print modell.predict(sms_spam_review_vector)

[ 0.52255854]


## Decision Tree

In [27]:
print modelt.predict(sms_spam_review_vector)

[ 0.]


## ------------------------------Conclusion-------------------------------
<li>As per the NB classifier new entry is the "ham' as it returned 0 
<li>As per the Random forest result is 0.28 which means it is near to 0 and lossing chances are more so we can say it is again '0'
<li> As per Liner regression new entry is a part of 'Spam' as its value is near to '1'
<li> As per Decision tree this entry is "ham'

## ----------------------------End of Assignment----------------------------------